# Weather Window Analysis

In [ ]:
import calendar

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets
from IPython.display import display, clear_output, HTML

from resourcecode import Client
from resourcecode.weatherwindow import compute_weather_windows


# define a few constant that will be used for the plots
MONTH_NAMES = list(calendar.month_name)
HOURS_BY_YEARS = np.arange(1, 744 + 1)

hs_access_threshold = np.linspace(1, 3, 5)
hs_ref = np.array([1, 2, 3])
time_ref = np.array([24, 48, 72])

client = Client()
weather_windows = {}

output = widgets.Output()
output_month_plots = widgets.Output()
output_event_plots = widgets.Output()

In [ ]:
def update_dataframe(change_url):
    global weather_windows
    with output:
        clear_output()
        print("The new selection is {new}".format(**change_url))
        print("Will re-compute the data end rerender the graphs")

    data = client.get_dataframe_from_url(change_url["new"], parameters=("hs",))
    # compute weather windows for each month
    weather_windows = {}
    for month in range(1, 13):
        weather_windows[month] = compute_weather_windows(
            data.hs, month=month, hs_access_threshold=hs_access_threshold
        )
    update_plots()


def update_plots(event=None):
    update_month_plots(event)
    update_events_plots(event)


def update_month_plots(event=None):
    with output_month_plots:
        clear_output()
        display(HTML("<h1>Weibull Distribution Parameters</h1>"))
        display(plot_weibull_distribution_parameters(month_widget.value))

        display(HTML("<h1>Mean Window Length</h1>"))
        display(plot_mean_window_length(month_widget.value))

        display(HTML("<h1>Probability of Occurrence</h1>"))
        display(plot_probability_of_occurrence(month_widget.value))


def update_events_plots(event=None):
    with output_event_plots:
        clear_output()
        display(HTML("<h1>Monthly evolution of number of events</h1>"))
        display(event_widget)
        display(plot_evolution_of_number_of_events(time_widget.value, hs_widget.value))

In [ ]:
url_widget = widgets.Text(
    value="",
    placeholder="Paste the URL of your selection",
    description="Selection:",
    disabled=False,
    continuous_update=False,
)

month_widget = widgets.Dropdown(
    options=MONTH_NAMES[1:],
    value=MONTH_NAMES[1],
    description="Month:",
)

time_widget = widgets.Dropdown(
    options=time_ref,
    value=time_ref[0],
    description="Time:",
)

hs_widget = widgets.Dropdown(
    options=hs_ref,
    value=hs_ref[0],
    description="Hs:",
)

event_widget = widgets.HBox([time_widget, hs_widget])

url_widget.observe(update_dataframe, "value")
month_widget.observe(update_plots, "value")
time_widget.observe(update_month_plots, "value")
hs_widget.observe(update_month_plots, "value")

display(url_widget)
display(output)
display(month_widget)
display(output_month_plots)
display(output_event_plots)

In [ ]:
# for each month, plot the weibull distribution result
def plot_weibull_distribution_parameters(month_name):
    month_index = MONTH_NAMES.index(month_name)
    weather_window_result = weather_windows[month_index]
    w = weather_window_result.weibull_distribution_result

    fig = make_subplots(rows=2, cols=1)
    fig.add_trace(
        go.Scatter(
            x=w.Ha,
            y=w.P,
            name="Fitted curve",
            text=f"x0 = {w.x0:0.5f}, b = {w.b:0.5f}, k = {w.k:0.5f}",
            showlegend=False,
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=w.Ha,
            y=w._MCFrHS,
            mode="markers",
            name="Experimental data",
            showlegend=False,
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(x=w._X, y=w._Y, mode="markers", showlegend=False), row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=w._X,
            y=np.log(np.log(1 / w.P)),
            showlegend=False,
            text=f"R = {w._residual:0.8f}",
        ),
        row=2,
        col=1,
    )

    fig.update_xaxes(title_text="Significant Wave Height (m)", row=1, col=1)
    fig.update_yaxes(title_text="Probability of Exceedance", row=1, col=1)

    fig.update_xaxes(
        title_text=r"$\ln\left(\ln\left(1 / P\right)\right)$", row=2, col=1
    )
    fig.update_yaxes(title_text=r"$\ln\left(H_s - x_0\right)$", row=2, col=1)
    fig.update_layout(
        title_text=(
            f"Weibull 3p fit − {month_name} (x0 = {w.x0:0.5f}, b = {w.b:0.5f}, k = {w.k:0.5f})"
        ),
        height=800,
    )
    return fig

In [ ]:
def plot_mean_window_length(month_name):
    month_index = MONTH_NAMES.index(month_name)

    weather_window_result = weather_windows[month_index]
    weibull_dist_result = weather_window_result.weibull_distribution_result

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=weibull_dist_result.Ha,
            y=weather_window_result.tau,
            showlegend=False,
        ),
    )
    fig.update_xaxes(
        title_text="Significant Wave Height (m)",
        range=[0, 10],
    )
    fig.update_yaxes(
        title_text="Mean window Lenght (Hours)",
        range=[0, 240],
    )

    fig.update_layout(
        title_text=f"Mean window Length − {month_name}",
        height=800,
    )
    return fig

In [ ]:
def plot_probability_of_occurrence(month_name):
    month_index = MONTH_NAMES.index(month_name)
    weather_window_result = weather_windows[month_index]

    fig = go.Figure()
    fig.add_traces(
        [
            go.Scatter(
                x=HOURS_BY_YEARS,
                y=weather_window_result.PT[i],
                name=f"{ha} m",
            )
            for i, ha in enumerate(hs_access_threshold)
        ]
    )

    fig.update_xaxes(
        title_text="Duration of Weather Window (Hours)",
    )
    fig.update_yaxes(
        title_text="Probability of Occurrence",
    )

    fig.update_layout(
        title_text=f"Probability of Occurrence − {month_name}",
        height=800,
    )
    return fig

In [ ]:
def plot_evolution_of_number_of_events(time, hs):
    fig = make_subplots(rows=3, cols=1)

    itime = np.where(HOURS_BY_YEARS == time)[0][0]
    iha = np.where(hs_access_threshold == hs)[0][0]

    nb_events = []
    nb_access_hours = []
    nb_waiting_hours = []
    for r in weather_windows.values():
        nb_events.append(r.number_events[iha, itime])
        nb_access_hours.append(r.number_access_hours[iha, itime])
        nb_waiting_hours.append(r.number_waiting_hours[iha, itime])

    fig.add_trace(
        go.Scatter(x=MONTH_NAMES[1:], y=nb_events, name="Number of events"),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=MONTH_NAMES[1:],
            y=nb_access_hours,
            name="Access time",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=MONTH_NAMES[1:],
            y=nb_waiting_hours,
            name="Waiting time",
        ),
        row=3,
        col=1,
    )

    fig.update_yaxes(title_text="Number of Events", row=1, col=1)
    fig.update_yaxes(title_text="Hours", row=2, col=1)
    fig.update_yaxes(title_text="Hours", row=3, col=1)
    fig.update_xaxes(title_text="Months", row=3, col=1)

    fig.update_layout(
        title_text=f"Weibull 3p fit − Hs = {hs}m, {time} hours window",
        height=1200,
    )

    return fig